In [1]:
import nltk 
import pandas as pd 
from nltk import ngrams
import numpy as np

In [285]:
annotatedData = pd.read_csv("../Data/PreProcessedYouTubeDataFileAndAnnotated.csv")

In [286]:
annotatedData = annotatedData.iloc[:,1:]

In [287]:
len(annotatedData)

2009

In [282]:
count = 0
print(len(annotatedData))
for index in range(len(annotatedData)) : 
    if annotatedData.iloc[index].get("Algerian Dialect") == "n":
        annotatedData.drop(annotatedData.index[index], inplace = True)
        count +=1
print(len(annotatedData))
print(count)

2000
2000
0


In [290]:
def remove_non_algerian_dialect(data):
    for index in range(len(data)) : 
        if data.iloc[index]["Algerian Dialect"] == "n":
            data.drop(data.index[index], inplace = True)
    return data 

In [293]:
annotatedData = remove_non_algerian_dialect(annotatedData)
print(len(annotatedData))

2000


In [294]:
annotatedData = annotatedData.iloc[:len(annotatedData),:6]

offensiveComment = pd.DataFrame(columns=['author', 'authorChannelUrl', 'text','likeCount', 'publishedAt', 'offensive/non offensive'])
nonOffensiveComment = pd.DataFrame(columns=['author', 'authorChannelUrl', 'text','likeCount', 'publishedAt', 'offensive/non offensive'])

for index in range(len(annotatedData)):
    if annotatedData.iloc[index]["offensive/non offensive"] == "p":
        offensiveComment = offensiveComment.append(annotatedData.iloc[index])
    else :
        nonOffensiveComment = nonOffensiveComment.append(annotatedData.iloc[index])


In [295]:
print(len(annotatedData))


2000


In [296]:
print(len(offensiveComment))
print(len(nonOffensiveComment))

1121
879


In [298]:
#data = annotatedData["offensive/non offensive"]
#print(data)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(annotatedData, annotatedData["offensive/non offensive"], test_size=0.35)

In [299]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1300, 6)
(700, 6)
(1300,)
(700,)


In [264]:
print(X_train.head(5))

             author                                   authorChannelUrl  \
1068   Sabri Skikda  http://www.youtube.com/channel/UCxg7d5rBW2PqzR...   
1314      Adam Reda  http://www.youtube.com/channel/UCp_A7wf_afDvOO...   
1832    Israa Haiba  http://www.youtube.com/channel/UC-fPpDcy6CktIg...   
1435  Femme Khedimi  http://www.youtube.com/channel/UCDNMwQpwG13f1F...   
1745  Walid Sanfour  http://www.youtube.com/channel/UCu6j0k2spmnaGp...   

                                                   text  likeCount  \
1068                      راحت عليك رجعتي تحلل كي ذليلة          0   
1314                       لا زين لا ضرافة وزيد قلة ادب          0   
1832  هاو نتي لابسا لمزير وتقولي تبلاوني شوفي يالغول...          0   
1435                            مكاش لي يبغيك الله غالب          0   
1745  يخي مهبول درت لمكياج داتها في رحها كون تنحيه ت...          0   

               publishedAt offensive/non offensive  
1068  2021-04-02T10:44:23Z                       p  
1314  2021-03-28T21:48:14Z  

In [265]:
for index in  range(5):
    comment = annotatedData.iloc[index]['text']
    for n in [1,2,3,4]:
        commentNgrams = ngrams(comment.split(), n)
        for grams in commentNgrams:
            print(' '.join(grams))

فمك
كبير
من
وجهك
تقبلي
في
كميرة
وجهك
صحيحة
فمك كبير
كبير من
من وجهك
وجهك تقبلي
تقبلي في
في كميرة
كميرة وجهك
وجهك صحيحة
فمك كبير من
كبير من وجهك
من وجهك تقبلي
وجهك تقبلي في
تقبلي في كميرة
في كميرة وجهك
كميرة وجهك صحيحة
فمك كبير من وجهك
كبير من وجهك تقبلي
من وجهك تقبلي في
وجهك تقبلي في كميرة
تقبلي في كميرة وجهك
في كميرة وجهك صحيحة
سلام
الله
عليكم
ورحمته
وبركاته
قلولهاةتزول
الكوة
كرستين
من
عنقها
سلام الله
الله عليكم
عليكم ورحمته
ورحمته وبركاته
وبركاته قلولهاةتزول
قلولهاةتزول الكوة
الكوة كرستين
كرستين من
من عنقها
سلام الله عليكم
الله عليكم ورحمته
عليكم ورحمته وبركاته
ورحمته وبركاته قلولهاةتزول
وبركاته قلولهاةتزول الكوة
قلولهاةتزول الكوة كرستين
الكوة كرستين من
كرستين من عنقها
سلام الله عليكم ورحمته
الله عليكم ورحمته وبركاته
عليكم ورحمته وبركاته قلولهاةتزول
ورحمته وبركاته قلولهاةتزول الكوة
وبركاته قلولهاةتزول الكوة كرستين
قلولهاةتزول الكوة كرستين من
الكوة كرستين من عنقها
افهمي
حاجة
النداء
يكون
بصفة
الجمع
لي
مش
معني
راح
يفهم
روحو
انو
مش
معني
الاخت
حنا
عارفين
خاوتنا
واباءنا
بلي
نخوة
ورجال
تاع


In [223]:
print(X_train["text"])

482     بلاك نتي تاني هدرتي فيها صح ولا لا مادام راكي ...
1255    بهلولة لما في بلدي اصحاب الرداءة يعطلهم قيمة و...
1201      مزالها دور فالقنوات تع زيقو اصلا هي من النفايات
319     تسريحة شعرك يا وسمك ديمودي بزاف فيقي شوي لروحك...
867     هذا باي حق تحكم على الناس كامل ب صورة وحدة كون...
                              ...                        
1262                                  الله لاتربحك نشالله
1895                                  الله يهديك و يهديهم
1858                                   صلى الله عليه وسلم
872     نتي بطريقتك هذي راك رادتها ماقالت حتى شيء هيا ...
1117                                     جبهتك والكل يضحك
Name: text, Length: 1500, dtype: object


In [300]:
def count_comments(result, comments, ys):
    '''
    Input:
        result: a dictionary that will be used to map each pair to its frequency
        comments: a list of comments
        ys: a list corresponding to the class of each comment ( 'p' if offensive 'n' otherwise )
    Output:
        result: a dictionary mapping each pair to its frequency
    '''
    
    for y, comment in zip(ys, comments):
        for n in [1,2,3,4]:
            commentNgrams = ngrams(comment.split(), n)
            for grams in commentNgrams:
                # define the key, which is the word and label tuple
                pair = (' '.join(grams) , y)

                # if the key exists in the dictionary, increment the count
                if pair in result:
                    result[pair] += 1

                # else, if the key is new, add it to the dictionary and set the count to 1
                else:
                    result[pair] = 1

    return result

In [301]:
freq = count_comments({},X_train["text"],y_train)

In [302]:
print(freq)

{('ه', 'n'): 40, ('حيرتوني', 'n'): 1, ('فيكم', 'n'): 2, ('نتوما', 'n'): 9, ('راني', 'n'): 10, ('مخللووععا', 'n'): 1, ('كفاه', 'n'): 2, ('دافعو', 'n'): 1, ('عليها', 'n'): 24, ('ه حيرتوني', 'n'): 1, ('حيرتوني فيكم', 'n'): 1, ('فيكم نتوما', 'n'): 1, ('نتوما راني', 'n'): 1, ('راني مخللووععا', 'n'): 1, ('مخللووععا كفاه', 'n'): 1, ('كفاه دافعو', 'n'): 1, ('دافعو عليها', 'n'): 1, ('ه حيرتوني فيكم', 'n'): 1, ('حيرتوني فيكم نتوما', 'n'): 1, ('فيكم نتوما راني', 'n'): 1, ('نتوما راني مخللووععا', 'n'): 1, ('راني مخللووععا كفاه', 'n'): 1, ('مخللووععا كفاه دافعو', 'n'): 1, ('كفاه دافعو عليها', 'n'): 1, ('ه حيرتوني فيكم نتوما', 'n'): 1, ('حيرتوني فيكم نتوما راني', 'n'): 1, ('فيكم نتوما راني مخللووععا', 'n'): 1, ('نتوما راني مخللووععا كفاه', 'n'): 1, ('راني مخللووععا كفاه دافعو', 'n'): 1, ('مخللووععا كفاه دافعو عليها', 'n'): 1, ('لا', 'p'): 49, ('بنتي', 'p'): 4, ('ماكانش', 'p'): 2, ('اللي', 'p'): 16, ('استشفى', 'p'): 1, ('الناس', 'p'): 23, ('كلها', 'p'): 1, ('غاضتها', 'p'): 1, ('شيماء', 'p'): 10, ('ها

In [303]:
print(len(freq))

52364


In [304]:
def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: dictionary from (ngram, label) to how often the ngram appears
        train_x: a list of comments
        train_y: a list of labels correponding to the comments ('p','n')
    Output:
        logprior: the log prior.
        loglikelihood: the log likelihood of you Naive bayes equation.
    '''
    loglikelihood = {}
    logprior = 0

 
    # calculate V, the number of unique ngrams in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # calculate N_pos and N_neg
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # if the label is positive (greater than zero)
        if pair[1] == "p":

            # Increment the number of offensive ngrams by the count for this (ngram, label) pair
            N_pos += freqs.get(pair,0)

        # else, the label is non offensive
        else:

            # increment the number of non offensive ngrams by the count for this (ngram,label) pair
            N_neg += freqs.get(pair,0)

    # Calculate D, the number of documents
    D = len(train_y)

    # Calculate D_pos, the number of positive documents (*hint: use sum(<np_array>))
    #D_pos = np.sum(train_y)
    D_pos = 0
    for index in range(len(train_y)) :
        if train_y.iloc[index] == "p":
            D_pos += 1

    # Calculate D_neg, the number of negative documents (*hint: compute using D and D_pos)
    D_neg = D - D_pos

    # Calculate logprior
    logprior = np.log(D_pos/D_neg)

    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = freqs.get((word,"p"),0)
        freq_neg = freqs.get((word,"n"),0)

        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos + 1 ) / (D_pos + V)
        p_w_neg = (freq_neg + 1 ) / (D_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos) - np.log(p_w_neg)

    ### END CODE HERE ###

    return logprior, loglikelihood


In [305]:
# You do not have to input any code in this cell, but it is relevant to grading, so please do not change anything
logprior, loglikelihood = train_naive_bayes(freq, X_train, y_train)
print(logprior)

#print(freqs.get(("diploma",0),0))
print(len(loglikelihood))

0.30070718361048254
50846


In [306]:
def naive_bayes_predict(comment, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the comment (if found in the dictionary) + logprior (a number)

    '''
    # initialize probability to zero
    p = 0

    # add the logprior
    p += logprior 

    for word in comment.split(' '):

        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood.get(word)

    return p


In [313]:
my_comment = ""
p = naive_bayes_predict(my_comment, logprior, loglikelihood)
print('The expected output is', p)

The expected output is 0.29693989625914174


In [314]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Input:
        test_x: A list of comments
        test_y: the corresponding labels for the list of comments
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of comments classified correctly)/(total # of comments)
    """
    accuracy = 0  # return this properly

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    y_hats = []
    for index in range(len(test_x)):
        comment = test_x.iloc[index]["text"]
        
        for index in range(len(test_y)):
            if test_y.iloc[index] == "n":
                test_y.iloc[index] = 0
            else :
                test_y.iloc[index] = 1
        
        # if the prediction is > 0
        if naive_bayes_predict(comment, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.absolute(y_hats - test_y))

    # Accuracy is 1 minus the error
    accuracy = 1- error

    ### END CODE HERE ###

    return accuracy


In [315]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(X_test, y_test, logprior, loglikelihood)))

Naive Bayes accuracy = 0.7629


In [ ]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))